# Detectron2: Fishial 


In [ ]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!gcc --version
!pip install imantics
# opencv is pre-installed on colab
# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
!pip install git+https://github.com/aleju/imgaug.git

##Connect google drive

In [ ]:
import torch, torchvision
torch.__version__
from google.colab import drive
drive.mount('/content/drive')

## Import dependences

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
!rm -rf output/

Before we can start training our model we need to download some dataset. We use our dataset which we put to google drive and unpacking on googlecolab VM

In [ ]:
!ls
!unzip 'drive/My Drive/dataset/fishial.zip' > /dev/null
!ls

Register our dataset to detectron2 as a COCO formate

In [ ]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools
from detectron2.data.datasets import register_coco_instances
from detectron2.data.datasets.coco import convert_to_coco_json
import cv2
# write a function that loads the dataset into detectron2's standard format
def get_dataset_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_region_data.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for _, v in imgs_anns.items():
        if len(v["regions"]['0']['shape_attributes']['all_points_x']) < 20:
          continue
        record = {}
        filename = os.path.join(img_dir, v["filename"])
        width, height = cv2.imread(filename).shape[:2]        
        record["file_name"] = v["filename"]
        record["height"] = width 
        record["width"] = height
      
        annos = v["regions"]
        objs = []
        for _, anno in annos.items():
            assert not anno["region_attributes"]
            anno = anno["shape_attributes"]
            
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = list(itertools.chain.from_iterable(poly))

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

from detectron2.data import DatasetCatalog, MetadataCatalog

for d in ["val"]:
    DatasetCatalog.register("fishial/" + d, lambda d=d: get_dataset_dicts("fishial/" + d))
    MetadataCatalog.get("fishial/" + d).set(thing_classes=["fishial"])
    convert_to_coco_json("fishial/" + d, 'fishial/'+d+'.json')

register_coco_instances("fishial/train-coco", {}, "fishial/train.json",
                        "fishial/train")

register_coco_instances("fishial/val-coco", {}, "fishial/val.json",
                        "fishial/val")

# balloon_metadata_train = MetadataCatalog.get("fishial/train-coco")
balloon_metadata_val = MetadataCatalog.get("fishial/val-coco")


Config train pipline

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.evaluation.coco_evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer

class MyTrainer(DefaultTrainer):

  # Uncomment if you want to made evalute VAL sets on COCOEvalutor
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = r"drive/My Drive/dataset/output/new29.230-42.681/model_final.pth"

cfg.DATASETS.TRAIN = ("fishial/train-coco",)
cfg.DATASETS.TEST = ("fishial/val-coco",)
# this parameter evaluate model on validation dataset each 2000 steps 
cfg.TEST.EVAL_PERIOD = 2000
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00018
cfg.SOLVER.MAX_ITER = 25000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)

# If you useing caustom data loader
# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = MyTrainer(cfg) 
# trainer.resume_or_load(resume=False)
# trainer.train()

In [ ]:
# copy checkpoint to google drive 
!mkdir -p 'drive/My Drive/dataset/output/newFaster28.379-41'
!cp -R 'output/model_final.pth' 'drive/My Drive/dataset/output/newFaster28.379-41'

##Creating a predictor object.

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
import urllib
import os
cfg.MODEL.WEIGHTS = r'/content/drive/My Drive/Colab Notebooks/TEST/model_0059999.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.19   # set the testing threshold for this model
cfg.DATASETS.TEST = ("fishial/val", )
predictor = DefaultPredictor(cfg)

###Visualization for Instance segmentation Model

In [ ]:
dataset_dicts = get_dataset_dicts("fishial/val")
import time
for d in random.sample(dataset_dicts, 20):
    print(d['file_name'])
    im = cv2.imread(os.path.join('fishial/val', d['file_name']))
    start_time = time.time()
    outputs = predictor(im)
    print("--- %s seconds ---" % (time.time() - start_time))
    v = Visualizer(im[:, :, ::-1],
                   MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), 
                   scale=4, 
                    instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
urllib.request.urlretrieve("https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT9Voxjof7LSZyIMzHUBar-SI9TIm3RYYSxzr7qc19X-HSHJ4h1&usqp=CAU", "00000001.jpg")
im = cv2.imread( "00000001.jpg")
outputs = predictor(im)
print(len(outputs['instances']))
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=4, 
               instance_mode=ColorMode.IMAGE_BW)   # remove the colors of unsegmented pixels
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (14, 10))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
masks = np.array(outputs['instances'].get('pred_masks')[0].to('cpu'))
print(masks)

###For Bouding box

In [ ]:
dataset_dicts = get_dataset_dicts("fishial/val")
for d in random.sample(dataset_dicts, 3):
    urllib.request.urlretrieve("https://www.aqvium.ru/images/blog/2016-11-08-glofish/Origins-glofish.jpg", "00000001.jpg")
    im = cv2.imread(d['file_name'])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=balloon_metadata, scale=0.8)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
urllib.request.urlretrieve("https://www.aqvium.ru/images/blog/2016-11-08-glofish/Origins-glofish.jpg", "00000001.jpg")
im = cv2.imread(d['file_name'])
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1], metadata=balloon_metadata, scale=0.8)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (14, 10))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()

If we want we can also display the training process through Tensorboard.

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
import json
import matplotlib.pyplot as plt

experiment_folder = './output'

def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines

experiment_metrics = load_json_arr(experiment_folder + '/metrics.json')

plt.plot(
    [x['iteration'] for x in experiment_metrics], 
    [x['total_loss'] for x in experiment_metrics])
plt.plot(
    [x['iteration'] for x in experiment_metrics if 'validation_loss' in x], 
    [x['validation_loss'] for x in experiment_metrics if 'validation_loss' in x])
plt.legend(['total_loss', 'validation_loss'], loc='upper left')
plt.show()

In [ ]:
!rm -rf output/

In [ ]:
# This is the video we're going to process
from IPython.display import YouTubeVideo, display
video = YouTubeVideo("H3UwLsvACl4", width=500)
display(video)

In [ ]:
# Install dependencies, download the video, and crop 5 seconds for processing
!pip install youtube-dl
!pip uninstall -y opencv-python opencv-contrib-python
!apt install python3-opencv  # the one pre-installed have some issues
!youtube-dl https://www.youtube.com/watch?v=H3UwLsvACl4 -f 22 -o video.mp4
!ffmpeg -i video.mp4 -t 00:00:06 -c:v copy video-clip.mp4

In [ ]:
# Run frame-by-frame inference demo on this video (takes 3-4 minutes)
# Using a model trained on COCO dataset
# !git clone https://github.com/facebookresearch/detectron2
!python detectron2/demo/demo.py --config-file detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml --video-input video-clip.mp4 --confidence-threshold 0.1 --output video-output.mkv \
  --opts MODEL.WEIGHTS output/model_final.pth

In [ ]:
# Download the results
from google.colab import files
files.download('via_region_data.json')

In [ ]:
def get_format_dict(title, area, array_x, array_y, filename):
    return {
            title: {
                "fileref": "",
                "size": area,
                "filename": filename,
                "base64_img_data": "",
                "verified": False,
                "correct": False,
                "file_attributes": {
                },
                "regions": {
                    "0": {
                        "shape_attributes": {
                            "name": "polygon",
                            "all_points_x": array_x,
                            "all_points_y": array_y
                        },
                        "region_attributes": {

                        }
                    }
                }
            }
        }

##Create polygons dictionary
this code for create synthetic dataset

In [ ]:
import os, json
import cv2
import pandas as pd
from shutil import copyfile
import re
from imantics import Polygons, Mask
import warnings

path_img = r'parsing'
list_path_img = [os.path.basename(i) for i in os.listdir(path_img)]
anotation_dict = {}
for idx, img_name in enumerate(list_path_img):
  print("Score: {}".format(len(list_path_img) - idx), len(anotation_dict))
  im = cv2.imread(os.path.join(path_img, img_name))
  w,h,_ = im.shape
  outputs = predictor(im)
  for z in range(len(outputs['instances'])):
    masks = np.array(outputs['instances'].get('pred_masks')[z].to('cpu'))
    polygons = Mask(masks).polygons()
    x_array = []
    y_array = []
    for i in polygons.points[0]:
      x_array.append(int(i[0]))
      y_array.append(int(i[1]))

    anotation_dict.update(get_format_dict(
        img_name+"_main_"+str(z),
        w*h,
        x_array,
        y_array,
        img_name))
    
with open('via_region_data.json', 'w') as fp:
    json.dump(anotation_dict, fp)